# SOLUCIÓN EJERCICIO RNN

### LIBRERIAS Y CARGA DE DATASET

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense

# Carga de datos
data = pd.read_csv('household_power_consumption.txt', sep=';', parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, low_memory=False, na_values=['nan','?'], index_col='dt')

# Limpieza de datos
data = data.dropna()

### PREPROCESADO DE DATOS

In [ ]:
# Normalización
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Global_active_power']])

# Preparación de secuencias
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length), 0]
        y = data[i + seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 60 # Ejemplo de longitud de secuencia
X, y = create_sequences(scaled_data, seq_length)

# División en conjuntos de entrenamiento y prueba
train_size = int(len(X) * 0.8)
X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

# Reformateo para LSTM/GRU
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

### LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_lstm.add(LSTM(50))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predicciones LSTM
lstm_predictions_scaled = model_lstm.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1))

### GRU

In [ ]:
model_gru = Sequential()
model_gru.add(GRU(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_gru.add(GRU(50))
model_gru.add(Dense(1))
model_gru.compile(optimizer='adam', loss='mse')
model_gru.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Predicciones GRU
gru_predictions_scaled = model_gru.predict(X_test)
gru_predictions = scaler.inverse_transform(gru_predictions_scaled)

### COMPARATIVA

In [ ]:
# Cálculo de métricas
lstm_rmse = np.sqrt(mean_squared_error(y_test_original, lstm_predictions))
lstm_mae = mean_absolute_error(y_test_original, lstm_predictions)

gru_rmse = np.sqrt(mean_squared_error(y_test_original, gru_predictions))
gru_mae = mean_absolute_error(y_test_original, gru_predictions)

# Visualización
plt.figure(figsize=(14, 6))
plt.plot(y_test_original, label='Real')
plt.plot(lstm_predictions, label='LSTM', alpha=0.7)
plt.plot(gru_predictions, label='GRU', alpha=0.7)
plt.legend()
plt.show()

# Tabla comparativa
print(pd.DataFrame({'Metric': ['RMSE', 'MAE'], 'LSTM': [lstm_rmse, lstm_mae], 'GRU': [gru_rmse, gru_mae]}))